In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import datetime
import time
from time import strftime, gmtime

In [2]:
%matplotlib inline
import statsmodels.api as sm
import matplotlib
from sklearn.feature_selection import RFE

In [3]:
import pandas as pd
df_flights = pd.read_csv("C:/Users/b_min/Desktop/Capstone/FlightDataClean.csv")

In [4]:
#df_flights.head()

In [5]:
df_flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
Date                    object
DAY_OF_WEEK             int64
AIRLINE                 object
FLIGHT_NUMBER           int64
TAIL_NUMBER             object
ORIGIN_AIRPORT          object
DESTINATION_AIRPORT     object
SCHEDULED_DEPARTURE     int64
DEPARTURE_TIME          float64
DEPARTURE_DELAY         float64
TAXI_OUT                float64
WHEELS_OFF              float64
SCHEDULED_TIME          float64
ELAPSED_TIME            float64
AIR_TIME                float64
DISTANCE                int64
WHEELS_ON               float64
TAXI_IN                 float64
SCHEDULED_ARRIVAL       int64
ARRIVAL_TIME            float64
ARRIVAL_DELAY           float64
DIVERTED                int64
CANCELLED               int64
CANCELLATION_REASON     object
AIR_SYSTEM_DELAY        float64
SECURITY_DELAY          float64
AIRLINE_DELAY           float64
LATE_AIRCRAFT_DELAY     float64
WEATHER_DEL

DEPARTURE_DELAY (float) = SCHEDULED_DEPARTURE - DEPARTURE_TIME(both are of type object)--Technically we don't need these 2 cols 
ARRIVAL_DELAY (float) = SCHEDULED_ARRIVAL - ARRIVAL_TIME(both are of type object)--Technically we don't need these 2 cols

WHEELS_ON and WHEELS_OFF are of type object too but they cannot be calculated, therefore they should be converted to int/float.
I converted the time format into minutes(using formula cell*1440) so that it can be converted to integer. I did this in Excel. 

Because of the formula i could not delete the wheels_on and wheels_off column of object data type. So I opened a new notebook and dropped it in Python. Now we have the correct format of data types. so I saved the dataset. and applied XGBOOST in a new notebook. 

Converting time into the correct format.

In [6]:
# converting input time value to datetime.
def conv_time(time_val):
    if pd.isnull(time_val):
        return np.nan
    else:
            # replace 24:00 o'clock with 00:00 o'clock:
        if time_val == 2400: time_val = 0
            # creating a 4 digit value out of input value:
        time_val = "{0:04d}".format(int(time_val))
            # creating a time datatype out of input value: 
        time_formatted = datetime.time(int(time_val[0:2]), int(time_val[2:4]))
    return time_formatted

In [7]:
df_flights['ARRIVAL_TIME'] = df_flights['ARRIVAL_TIME'].apply(conv_time)
df_flights['DEPARTURE_TIME'] = df_flights['DEPARTURE_TIME'].apply(conv_time)
df_flights['SCHEDULED_DEPARTURE'] = df_flights['SCHEDULED_DEPARTURE'].apply(conv_time)
df_flights['WHEELS_OFF'] = df_flights['WHEELS_OFF'].apply(conv_time)
df_flights['WHEELS_ON'] = df_flights['WHEELS_ON'].apply(conv_time)
df_flights['SCHEDULED_ARRIVAL'] = df_flights['SCHEDULED_ARRIVAL'].apply(conv_time)

In [8]:
df_flights['WHEELS_ON'] = df_flights['WHEELS_ON'].astype(str)
df_flights['WHEELS_OFF'] = df_flights['WHEELS_OFF'].astype(str)

In [9]:
def time_to_minutes(time_str):  
    try:  
        hours, minutes, seconds = time_str.split(':')  
    except ValueError:  
        return -1  
    return int(hours)*60 + int(minutes) + int(seconds)/60.0  

In [10]:
df_flights['WHEELS_OFF'] = df_flights['WHEELS_OFF'].apply(time_to_minutes)

In [11]:
df_flights['WHEELS_ON'] = df_flights['WHEELS_ON'].apply(time_to_minutes)

In [12]:
df_flights[['WHEELS_OFF','WHEELS_ON']]

,WHEELS_OFF,WHEELS_ON
0,15.0,244.0
1,14.0,457.0
2,34.0,480.0
3,30.0,468.0
4,35.0,174.0
...,...,...
5819074,17.0,469.0
5819075,12.0,267.0
5819076,7.0,264.0
5819077,3.0,207.0


In [13]:
df_flights.head(1)

,Date,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,...,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,Delayed or Cancelled,Delayed or Late
0,2015-01-01,4,AS,98,N407AS,ANC,SEA,00:05:00,23:54:00,-11.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,-22.0


In [14]:
df_flights.isnull().sum()

Date                          0
DAY_OF_WEEK                   0
AIRLINE                       0
FLIGHT_NUMBER                 0
TAIL_NUMBER               14721
ORIGIN_AIRPORT                0
DESTINATION_AIRPORT           0
SCHEDULED_DEPARTURE           0
DEPARTURE_TIME            86153
DEPARTURE_DELAY           86153
TAXI_OUT                  89047
WHEELS_OFF                    0
SCHEDULED_TIME                6
ELAPSED_TIME             105071
AIR_TIME                 105071
DISTANCE                      0
WHEELS_ON                     0
TAXI_IN                   92513
SCHEDULED_ARRIVAL             0
ARRIVAL_TIME              92513
ARRIVAL_DELAY            105071
DIVERTED                      0
CANCELLED                     0
CANCELLATION_REASON     5729195
AIR_SYSTEM_DELAY        4755640
SECURITY_DELAY          4755640
AIRLINE_DELAY           4755640
LATE_AIRCRAFT_DELAY     4755640
WEATHER_DELAY           4755640
Delayed or Cancelled          0
Delayed or Late          105071
dtype: i

CANCELLATION_REASON, AIR_SYSTEM_DELAY, SECURITY_DELAY, AIRLINE_DELAY, LATE_AIRCRAFT_DELAY, WEATHER_DELAY
are the columns with maximum number of missing values. However, the missing values are present not because of the bad quality of data but due to the fact that these instances never occurred. Hence we can simply subsitute 0 in the empty cells.

In [15]:
df_flights['AIRLINE_DELAY'] = df_flights['AIRLINE_DELAY'].fillna(0)
df_flights['AIR_SYSTEM_DELAY'] = df_flights['AIR_SYSTEM_DELAY'].fillna(0)
df_flights['SECURITY_DELAY'] = df_flights['SECURITY_DELAY'].fillna(0)
df_flights['LATE_AIRCRAFT_DELAY'] = df_flights['LATE_AIRCRAFT_DELAY'].fillna(0)
df_flights['WEATHER_DELAY'] = df_flights['WEATHER_DELAY'].fillna(0)

CANCELLATION_REASON is a categorical variables wherein<br>
A - Airline/Carrier<br>
B - Weather<br>
C - National Air System<br>
D - Security<br>

In [16]:
df_flights['CANCELLATION_REASON'].value_counts()

B    48851
A    25262
C    15749
D       22
Name: CANCELLATION_REASON, dtype: int64

B - Weather is the main reason for cancellation.

In [17]:
# converting categoric value to numeric
df_flights.loc[df_flights['CANCELLATION_REASON'] == 'A', 'CANCELLATION_REASON'] = 1
df_flights.loc[df_flights['CANCELLATION_REASON'] == 'B', 'CANCELLATION_REASON'] = 2
df_flights.loc[df_flights['CANCELLATION_REASON'] == 'C', 'CANCELLATION_REASON'] = 3
df_flights.loc[df_flights['CANCELLATION_REASON'] == 'D', 'CANCELLATION_REASON'] = 4

# -----------------------------------
# converting NaN data to numeric zero
df_flights['CANCELLATION_REASON'] = df_flights['CANCELLATION_REASON'].fillna(0)

Many of the variables still have missing values. However these values can be calculated from other columns. For example, arrival delay can be calculated by subtracting arrival time from scheduled arrival. However, the columns that help perform the calculation itself have missing values. Hence, the dependent column, which is arrival_delay in this case, cannot be calucated. A similar approach can used for other columns. Therefore these variables with missing values are considered as outliers and dropped.

In [18]:
df_flights = df_flights.dropna(axis=0)

In [19]:
#df_flights.to_csv(r'C:/Users/b_min/Desktop/Capstone/finallycleansedflights.csv')